In [1]:
%load core.py

explore


In [1]:
# %load core.py

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow import keras
from keras import Sequential 

import src.utils as utils
import src.tfmodel as tfmodel
import src.vis as vis

"""
Practice project - histology

"""
#-----------------------------------
#Vars
#-----------------------------------

CONFIG_FILE='config.yaml'
DSETNAME='patch_camelyon'
#-----------------------------------
#INITIALISE
#-----------------------------------

config=utils.readcfg(CONFIG_FILE)

wdirname=config['wdirname']
odirname=config['odirname']
batchlen=config['batchlen']

spath, wdir, odir, checkpoint_path, checkpoint_dir = utils.initialise(config)

#-----------------------------------
#MAIN START
#-----------------------------------

dtrain, dvalidation, dtest, dsinfo = utils.datainit(config, DSETNAME)

timg, tlabels, vimg, vlabels, testimg, testlabels = utils.tensorinit(dtrain, dvalidation, dtest)

#check shapes for both
#should correspond to batch size
print("labelshape:",vlabels.shape)
print("imgshape:",timg.shape)
print("batch size:",batchlen)

if config['PREPLOT']:
    vis.preplot(config, timg, tlabels)

#we are overfitting pretty heavily, try regularisation
l2reg=tf.keras.regularizers.L2(config['lamda'])
#kernel_regularizer=l2reg
  #not great, slow and not a big improvement, leave it out for now


model = tfmodel.build(config)

model, fitlog = tfmodel.train(model, timg, tlabels, vimg, vlabels, config, checkpoint_path)

vis.layerplot(config, model, timg, tlabels, odir)

print("CLEAN EXIT")
exit()

"""
performance log

3 layers 256 128, batch 512 - 0.6 vacc
4 layers 256 128, batch 512 - 0.63 vacc
4 layers 256 128 batch 8k  - ~0.65 vacc - v slow

4+1 batch 512               -   ~0.7 vacc
  1 conv2D 32,3             - seems unstable, vacc varying 0.65-0.75
  4 dense 256,128,56,12 
    4250 ms/step on CPU
    ~27 ms/step on GPU

try L2_regularisation on all layers
  - much slower (1.2 sec), still 0.7-0.75 vacc
  - maybe more stable but seems not worth

    Ok here's an example
    https://medium.com/analytics-vidhya/deep-learning-tutorial-patch-camelyon-data-set-d0da9034550e
      6 conv2D layers + 6 "maxpool" layers - what is this?
      3 dense layers
      gets ~0.75, 150ms/step

try adding this maxpool layer  - 0.72 vacc, more stable?
  faster - 17ms/step


up to three conv+maxpool layers
  256 1024 256 
  touches 0.77 but very slow to train





"""

"""
resources

really good explanation of maxpooling here
https://www.youtube.com/watch?v=ZjM_XQa5s6s

also nice CNN overview
https://www.youtube.com/watch?v=YRhxdVk_sIs

"""

2022-10-05 22:19:46.577562: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-10-05 22:19:47.935412: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.0de2b4431c6572ee74152a7ee0cd3fb1534e4a95.so
2022-10-05 22:19:47.935563: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2022-10-05 22:19:47.946400: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2022-10-05 22:19:49.634824: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.
/home/lachlan/.pyenv/versions/venv_patchcam/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


script path: /home/lachlan/CODEBASE/Patch_camelyon
working path: /home/lachlan/CODEBASE/Patch_camelyon/train
output path: /home/lachlan/CODEBASE/Patch_camelyon/out
cp: /home/lachlan/CODEBASE/Patch_camelyon/train/cp-{epoch:04d}.ckpt
cp: /home/lachlan/CODEBASE/Patch_camelyon/train
---------------
python version: 3.7.13 (default, Sep  4 2022, 13:19:50) 
[GCC 9.4.0]
tensorflow version: 2.9.2
GPU: True
---------------


2022-10-05 22:19:50.408228: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-05 22:19:50.409328: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (NVIDIA GeForce RTX 2070)
2022-10-05 22:19:51.082836: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-05 22:19:51.082881: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2022-10-05 22:19:51.082902: I tensorflow/core/common_runtime/pluggable_device/pluggable

labelshape: (256,)
imgshape: (256, 96, 96, 3)
batch size: 256


TypeError: preplot() missing 2 required positional arguments: 'timg' and 'tlabels'